In [ ]:
# импортируем все необходимые библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

# Идея 1 Предсказание уровней воды на реке Амур

In [ ]:
# Описание данных
# https://github.com/sberbank-ai/no_flood_with_ai_aij2020/blob/main/desc/datasets_description_ru.md

# Описание задачи
# https://github.com/sberbank-ai/no_flood_with_ai_aij2020

# Сами данные 
# https://storage.yandexcloud.net/datasouls-ods/materials/c8b9bab3/datasets.zip

In [2]:
%%bash
curl -o datasets.zip "https://storage.yandexcloud.net/datasouls-ods/materials/c8b9bab3/datasets.zip" 
unzip -o -q datasets.zip
rm -f datasets.zip
rm -r __MACOSX

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  303M  100  303M    0     0  17.9M      0  0:00:16  0:00:16 --:--:-- 19.0M


Полный архив данных наблюдений на гидрологических постах сети Росгидромета за период с 1984 по 2018 гг. Содержит данные об уровнях воды, расходах, температуре воды, наблюдения за поверхностью воды (становление ледостава, вскрытие)

In [3]:
path_to_hydro = './datasets/hydro/'
files_hydro = ! ls -1 $path_to_hydro

1. Ежедневные значения уровня воды на гидропостах

In [4]:
files_hydro_daily = [f for f in files_hydro if '_daily' in f] 
files_hydro_daily[:10]

['05001_daily.csv',
 '05002_daily.csv',
 '05004_daily.csv',
 '05008_daily.csv',
 '05009_daily.csv',
 '05012_daily.csv',
 '05013_daily.csv',
 '05016_daily.csv',
 '05019_daily.csv',
 '05020_daily.csv']

In [ ]:
!pwd

/content


In [1]:
!ls

sample_data


In [ ]:
%%time
# Собираем в pandas dataframe 

daily = []
for filename in files_hydro_daily:
    tmp = pd.read_csv(path_to_hydro + filename, \
    encoding = 'cp1251', sep = ';', skiprows = 2, index_col =False, \
    names = ['date','stage_avg','stage_min','stage_max','temp','water_code'])
    tmp['id'] = filename.replace('_daily.csv', '')
    daily.append(tmp)

daily = pd.concat(daily)

CPU times: user 4.09 s, sys: 456 ms, total: 4.54 s
Wall time: 4.56 s


In [ ]:
daily.head()

,date,stage_avg,stage_min,stage_max,temp,water_code,id
0,01.01.2000,253,253,253,,16,05001
1,02.01.2000,260,260,260,,16,05001
2,03.01.2000,259,259,259,,16,05001
3,04.01.2000,252,252,252,,16,05001
4,05.01.2000,250,250,250,,16,05001


In [ ]:
daily.loc[daily['temp'] =='', 'temp'] = np.nan
daily.loc[daily['temp'] =='       ', 'temp'] = np.nan
daily.loc[daily['temp'] =='                      ', 'temp'] = np.nan

In [ ]:
# Приводим к нужным типам

daily['date'] = pd.to_datetime(daily['date'], format='%d.%m.%Y')

daily.loc[daily['stage_avg'] == '       ', 'stage_avg'] = np.nan ## иначе из string нельзя перевести в float
daily['stage_avg'] = daily['stage_avg'].astype('float16')

daily['stage_min'] = daily['stage_min'].astype('float16')

daily.loc[daily['stage_max'] =='****   ', 'stage_max'] = np.nan ## иначе из string нельзя перевести в float
daily['stage_max'] = daily['stage_max'].astype('float16')

daily.loc[daily['temp'] =='', 'temp'] = np.nan
daily.loc[daily['temp'] =='       ', 'temp'] = np.nan
daily.loc[daily['temp'] =='                      ', 'temp'] = np.nan

cond = (~daily['temp'].isnull())&daily['temp'].str.contains(',')
daily.loc[cond, 'temp']=daily[cond].temp.map(lambda x:x.split('   ')[1])
daily['temp'] = daily['temp'].astype('float16')

ValueError: could not convert string to float: ' 7.90   29            '

In [ ]:

# for i, row in daily[~daily['temp'].isnull()].drop_duplicates().iterrows():
#     try:
#         g = float(row['temp'])
#     except:
#         print('*'+row['temp']+'*', len(row['temp']))
# #         break


2. Периодические значения (раз в 5 дней) высоты снега и толщины льда на гидропостах

In [ ]:
files_ice = [f for f in files_hydro if '_daily' in f]

3. Среднесуточный расход воды

In [ ]:
files_disch_d = [f for f in files_hydro if '_disch_d' in f]

4. Месячные экстремумы расхода воды

In [ ]:
files_disch_m = [f for f in files_hydro if '_disch_m' in f]